In [1]:
from line_solver import *
import numpy as np
GlobalConstants.set_verbose(VerboseLevel.STD)

In [2]:
# Series of Fork-Join Networks
model = Network('model')

In [3]:
# Create series of fork-join stages
delay = Delay(model, 'Delay1')
queue1 = Queue(model,'Queue1',SchedStrategy.PS)
queue2 = Queue(model,'Queue2',SchedStrategy.PS)
fork = Fork(model,'Fork')
join = Join(model,'Join', fork)
queue3 = Queue(model,'Queue3',SchedStrategy.PS)
queue4 = Queue(model,'Queue4',SchedStrategy.PS)
fork2 = Fork(model,'Fork2')
join2 = Join(model,'Join2', fork2)

jobclass1 = ClosedClass(model, 'class1', 10, delay, 0)

In [4]:
# Service configurations
queue1.set_service(jobclass1, Exp(1.0))
queue2.set_service(jobclass1, Exp(1.0))
delay.set_service(jobclass1, Exp(0.5))
queue3.set_service(jobclass1, Exp(1.0))
queue4.set_service(jobclass1, Exp(1.0))

In [5]:
# Series fork-join routing
P = model.init_routing_matrix()

# First fork-join stage
P.set(jobclass1, jobclass1, delay, fork, 1.0)
P.set(jobclass1, jobclass1, fork, queue1, 1.0)
P.set(jobclass1, jobclass1, fork, queue2, 1.0)
P.set(jobclass1, jobclass1, queue1, join, 1.0)
P.set(jobclass1, jobclass1, queue2, join, 1.0)
# Between stages: join -> fork2
P.set(jobclass1, jobclass1, join, fork2, 1.0)
# Second fork-join stage
P.set(jobclass1, jobclass1, fork2, queue3, 1.0)
P.set(jobclass1, jobclass1, fork2, queue4, 1.0)
P.set(jobclass1, jobclass1, queue3, join2, 1.0)
P.set(jobclass1, jobclass1, queue4, join2, 1.0)
P.set(jobclass1, jobclass1, join2, delay, 1.0)

model.link(P)

In [6]:
# Solve with multiple methods
solvers = [SolverJMT(model, seed=23000), SolverMVA(model)]
for i, solver in enumerate(solvers):
    print(f'SOLVER {i+1}: {solver.get_name()}')
    avgTable = solver.get_avg_table()

SOLVER 1: SolverJMT
JMT Model: /tmp/workspace/jsim/3370124402616682040/jmodel.jsim
JMT [method: default, lang: java, env: 21.0.8] completed in 1.254740s.
  Station JobClass    QLen    Util   RespT  ResidT    ArvR    Tput
0  Delay1   class1  1.6255  1.6255  1.9812  1.9812  0.7941  0.7942
1  Queue1   class1  2.4695  0.7886  3.0255  1.5128  0.7942  0.7895
2  Queue2   class1  2.6214  0.7811  3.4269  1.7134  0.7942  0.7945
3    Join   class1  2.8113  0.0000  1.6344  1.6344  1.5847  0.7945
4  Queue3   class1  2.7476  0.8061  3.5097  1.7549  0.7945  0.7908
5  Queue4   class1  2.8230  0.8190  3.5903  1.7952  0.7945  0.7912
6   Join2   class1  3.1266  0.0000  1.9626  1.9626  1.5855  0.7941
SOLVER 2: SolverMVA
MVA analysis [method: default(egflin), lang: java, env: 21.0.8] completed in 0.057938s.
  Station JobClass    QLen    Util   RespT  ResidT    ArvR   Tput
0  Delay1   class1  1.5101  1.5101  2.0000  2.0000  0.7550  0.755
1  Queue1   class1  2.8654  0.7550  3.7951  1.8975  0.3775  0.755
2  Q